In [ ]:
import dash
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objs as go
import pandas as pd
import json

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})



import pandas as pd
df = pd.DataFrame(columns=['x_val','z_val','y_val'])
df.loc['a'] = [11.313449728149418,0.13039110880256777,0.5386387766748618]
df.loc['b'] = [11.321463427315383,0.2360697833061771,1.32441455152796]
df.loc['c'] = [10.127132005050942,0.23085014016641864,1.0961116175427044]
df.loc['d'] = [11.639819269465233,0.0958798324712593,0.6506370305953094]
df.loc['e'] = [8.892696370438149,0.08223988244819926,0.6440321391968353]
df.loc['f'] = [6.711586646011124,0.3657515974938044,0]
df.loc['g'] = [7.095030650760687,0,0.5723062047617504]
df.loc['h'] = [6.4523124528415,0,1.293852184258803]
df.loc['i'] = [7.165105300812886,0.4151365420301895,-0.5920674079031845]
df.loc['j'] = [7.480703395137295,0.14284429977557123,1.0600936940126982]
df.loc['k'] = [5.570775744372319,0,0]
df.loc['l'] = [4.358946555449826,0,0]

df2 = pd.DataFrame(columns=['x_val2','z_val2','y_val2'])
df2.loc['a'] = [9.313449728149418,0.13039110880256777,0.5386387766748618]
df2.loc['b'] = [1.321463427315383,0.2360697833061771,1.32441455152796]
df2.loc['c'] = [9.127132005050942,0.23085014016641864,1.0961116175427044]
df2.loc['d'] = [11.639819269465233,0.0958798324712593,0.56506370305953094]
df2.loc['e'] = [0.892696370438149,0.108223988244819926,0.6440321391968353]
df2.loc['f'] = [16.711586646011124,0.3657515974938044,0]
df2.loc['g'] = [7.095030650760687,0,0.5723062047617504]
df2.loc['h'] = [6.4523124528415,0,13.293852184258803]
df2.loc['i'] = [17.165105300812886,0.4151365420301895,-0.5920674079031845]
df2.loc['j'] = [7.480703395137295,0.14284429977557123,13.0600936940126982]
df2.loc['k'] = [5.570775744372319,2,0]
df2.loc['l'] = [4.358946555449826,0,3]

def create_figure(skip_points=[]):
    dfs = df.drop(skip_points)
    return px.scatter_3d(dfs, x = 'x_val', y = 'y_val', z = 'z_val')
f= create_figure()

def create_figure2(skip_points2=[]):
    dfs = df2.drop(skip_points2)
    return px.scatter_3d(dfs, x = 'x_val2', y = 'y_val2', z = 'z_val2')
d= create_figure2()

app.layout = html.Div(
                    [html.Button('Delete', id='delete'),
                    html.Button('Clear Selection', id='clear'),
                    html.Button('Procrustes analysis', id='procrustes_analyses'),
                    dcc.Graph(id = '3d_scat', figure=f),
                    html.Div('selected:'),
                    html.Div(id='selected_points'), #, style={'display': 'none'})),
                    html.Div('deleted:'),
                    html.Div(id='deleted_points'),

                    html.Button('Delete2', id='delete2'),
                    html.Button('Clear Selection2', id='clear2'),
                    dcc.Graph(id = '3d_scat2', figure=d),
                    html.Div('selected2:'),
                    html.Div(id='selected_points2'), #, style={'display': 'none'})),
                    html.Div('deleted2:'),
                    html.Div(id='deleted_points2')
                       #, style={'display': 'none'}))
])

@app.callback(
            Output('deleted_points', 'children'),
            Input('delete', 'n_clicks'),
            State('selected_points', 'children'),
            State('deleted_points', 'children')
 )

def delete_points(n_clicks, selected_points, delete_points):
    print('n_clicks:',n_clicks)
    if selected_points:
        selected_points = json.loads(selected_points)
    else:
        selected_points = []

    if delete_points:
        deleted_points = json.loads(delete_points)
    else:
        deleted_points = []
    ns = [p['pointNumber'] for p in selected_points]
    new_indices = [df.index[n] for n in ns if df.index[n] not in deleted_points]
    print('new',new_indices)
    deleted_points.extend(new_indices)
    return json.dumps(deleted_points)



@app.callback(
            Output('deleted_points2', 'children'),
            Input('delete2', 'n_clicks'),
            State('selected_points2', 'children'),
            State('deleted_points2', 'children')
 )

def delete_points2(n_clicks, selected_points2, delete_points2):
    print('n_clicks:',n_clicks)
    if selected_points2:
        selected_points2 = json.loads(selected_points2)
    else:
        selected_points2 = []

    if delete_points2:
        deleted_points2 = json.loads(delete_points2)
    else:
        deleted_points2 = []
    ns = [p['pointNumber'] for p in selected_points2]
    new_indices2 = [df.index[n] for n in ns if df.index[n] not in deleted_points2]
    print('new',new_indices2)
    deleted_points2.extend(new_indices2)
    return json.dumps(deleted_points2)



@app.callback(
            Output('selected_points', 'children'),
            Input('3d_scat', 'clickData'),
            Input('deleted_points', 'children'),
            Input('clear', 'n_clicks'),
            State('selected_points', 'children'))

def select_point(clickData, deleted_points, clear_clicked, selected_points):
    ctx = dash.callback_context
    ids = [c['prop_id'] for c in ctx.triggered]

    if selected_points:
        results = json.loads(selected_points)
    else:
        results = []


    if '3d_scat.clickData' in ids:
        if clickData:
            for p in clickData['points']:
                if p not in results:
                    results.append(p)
    if 'deleted_points.children' in ids or  'clear.n_clicks' in ids:
        results = []
    results = json.dumps(results)
    return results



@app.callback(
            Output('selected_points2', 'children'),
            Input('3d_scat2', 'clickData'),
            Input('deleted_points2', 'children'),
            Input('clear2', 'n_clicks'),
            State('selected_points2', 'children')
              )


def select_point2(clickData, deleted_points2, clear2_clicked, selected_points2):
    ctx = dash.callback_context
    ids = [c['prop_id'] for c in ctx.triggered]

    if selected_points2:
        results2 = json.loads(selected_points2)
    else:
        results2 = []


    if '3d_scat2.clickData' in ids:
        if clickData:
            for p in clickData['points']:
                if p not in results2:
                    results2.append(p)
    if 'deleted_points2.children' in ids or  'clear2.n_clicks' in ids:
        results2 = []
    results2 = json.dumps(results2)
    return results2

@app.callback(
            Output('3d_scat2', 'figure'),
            Input('selected_points2', 'children'),
            Input('deleted_points2', 'children'),
            State('deleted_points2', 'children'))

def chart_3d2( selected_points2, deleted_points2_input, deleted_points_state2):
    global d
    deleted_points2 = json.loads(deleted_points_state2) if deleted_points_state2 else []
    d = create_figure2(deleted_points2)

    selected_points2 = json.loads(selected_points2) if selected_points2 else []
    if selected_points2:
        d.add_trace(
            go.Scatter3d(
                mode='markers',
                x=[p['x'] for p in selected_points2],
                y=[p['y'] for p in selected_points2],
                z=[p['z'] for p in selected_points2],
                marker=dict(
                    color='green',
                    size=5,
                    line=dict(
                        color='green',
                        width=2
                    )
                ),
                showlegend=False
            )
        )

    return d

@app.callback(
            Output('3d_scat', 'figure'),
            Input('selected_points', 'children'),
            Input('deleted_points', 'children'),
            State('deleted_points', 'children'))

def chart_3d( selected_points, deleted_points_input, deleted_points_state):
    global f
    deleted_points = json.loads(deleted_points_state) if deleted_points_state else []
    f = create_figure(deleted_points)

    selected_points = json.loads(selected_points) if selected_points else []
    if selected_points:
        f.add_trace(
            go.Scatter3d(
                mode='markers',
                x=[p['x'] for p in selected_points],
                y=[p['y'] for p in selected_points],
                z=[p['z'] for p in selected_points],
                marker=dict(
                    color='red',
                    size=5,
                    line=dict(
                        color='red',
                        width=2
                    )
                ),
                showlegend=False
            )
        )
    return f

if __name__ == '__main__':
    app.run_server(debug=False, port=8049)



Dash is running on http://127.0.0.1:8049/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8049/ (Press CTRL+C to quit)
/home/maxi/anaconda3/envs/pythonProject1/lib/python3.9/site-packages/dash/resources.py:63: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources

127.0.0.1 - - [22/Sep/2021 17:30:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:09] "POST /_dash-update-component HTTP/1.1" 200 -


n_clicks: None
new []
n_clicks: None
new []


127.0.0.1 - - [22/Sep/2021 17:30:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.

n_clicks: 1
new ['a', 'a', 'l', 'b']


...

127.0.0.1 - - [22/Sep/2021 17:30:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.

n_clicks: 1
new ['g', 'e', 'b', 'c']


127.0.0.1 - - [22/Sep/2021 17:30:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:58] "GET /_dash-component-suites/dash_renderer/react@16.v1_1_2m1576595738.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:58] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_1_2m1576595738.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:58] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_1_2m1576595738.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:58] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_1m1576596177.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:58] "GET /_dash-component-suites/dash_core_components/highlight.v1_3_1m1576595950.pack.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:59] "GET /_dash-component-suites/dash_renderer/dash_renderer.v1_1_2m1576595738.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 17:30:59] "GET /_dash-component-suites/dash_core_components/dash_core_component

n_clicks:n_clicks: None
new []
 None
new []


127.0.0.1 - - [22/Sep/2021 17:31:02] "POST /_dash-update-component HTTP/1.1" 200 -
